# Time Dependent Evolution

In [2]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from quantum_tools import *
from experimental import *
from spam import *
from timedependent_channels import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=5, suppress=True)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Interacting Hamiltonian

In [ ]:
n = 2
d = 2**n

tf.random.set_seed(42)
np.random.seed(42)

H = SpinSpin()

channel = MagnusPropagator(liouvillian=H, grid_size=10)
state = np.zeros((d,d))
state[1,1] = 1.
state = tf.cast(state, dtype = precision)

state = [state, np.pi/4]
state1 = channel.apply_channel(state)
print(state1)

In [ ]:
n = 2
d = 2**n


tf.random.set_seed(42)
np.random.seed(42)

H = SpinSpin()

channel = MagnusPropagator(liouvillian=H, grid_size=100)
state = np.zeros((d,d))
state[1,1] = 1.
state = tf.cast(state, dtype = precision)

state = [state, np.pi/4]
state2 = channel.apply_channel(state)
print(state2)

In [ ]:
n = 2
d = 2**n


tf.random.set_seed(42)
np.random.seed(42)

H = SpinSpin()

channel = MagnusPropagator(liouvillian=H, grid_size=1000)
state = np.zeros((d,d))
state[1,1] = 1.
state = tf.cast(state, dtype = precision)

state = [state, np.pi/4]
state3 = channel.apply_channel(state)
print(state3)

In [ ]:
print(state1)
print(state2)
print(state3)

In [10]:
n = 2
d = 2**n
T = 0.05

tf.random.set_seed(42)
np.random.seed(42)

H_target = SpinSpin()
H_model = SpinSpin()

channel_target = MagnusPropagator(liouvillian=H_target, grid_size=100)
channel_model = MagnusPropagator(liouvillian=H_model, grid_size=100)
print(state_fidelity(channel_target.choi(T), channel_model.choi(T))/16)

tf.Tensor(0.9678158706212285, shape=(), dtype=float64)


In [11]:
T = 0.01
optimizer = tf.optimizers.Adam(learning_rate=0.1)

for i in tqdm(range(50)):
    with tf.GradientTape() as tape:
        tape.watch(channel_model.parameter_list)
        choi_target = channel_target.choi(T)
        choi_model = channel_model.choi(T)
        loss = tf.math.reduce_sum(tf.abs(choi_model - choi_target) ** 2)
    
    print(loss)
    gradients = tape.gradient(loss, H_model.parameter_list)
    optimizer.apply_gradients(zip(gradients, H_model.parameter_list))


    

  0%|          | 0/50 [00:00<?, ?it/s]

tf.Tensor(0.041980510352296205, shape=(), dtype=float64)
tf.Tensor(0.03153378910415431, shape=(), dtype=float64)
tf.Tensor(0.022601050537356355, shape=(), dtype=float64)
tf.Tensor(0.01589844561388736, shape=(), dtype=float64)
tf.Tensor(0.011070326460466701, shape=(), dtype=float64)
tf.Tensor(0.007711375981659899, shape=(), dtype=float64)
tf.Tensor(0.005582870899846097, shape=(), dtype=float64)
tf.Tensor(0.004421527907266697, shape=(), dtype=float64)
tf.Tensor(0.003925939299894242, shape=(), dtype=float64)
tf.Tensor(0.003791788905127238, shape=(), dtype=float64)
tf.Tensor(0.003798993939160887, shape=(), dtype=float64)
tf.Tensor(0.0038328489870756567, shape=(), dtype=float64)
tf.Tensor(0.0038384809793241713, shape=(), dtype=float64)
tf.Tensor(0.0037774992658574777, shape=(), dtype=float64)
tf.Tensor(0.0036115527625520203, shape=(), dtype=float64)
tf.Tensor(0.0033195523590990412, shape=(), dtype=float64)
tf.Tensor(0.0029173192042725865, shape=(), dtype=float64)
tf.Tensor(0.002449901985024

In [9]:
channel_target = MagnusPropagator(liouvillian=H_target, grid_size=100)
channel_model = MagnusPropagator(liouvillian=H_model, grid_size=100)
print(state_fidelity(channel_target.choi(T), channel_model.choi(T))/16)

print([np.array(param) for param in channel_model.parameter_list])
print([np.array(param) for param in channel_target.parameter_list])

[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.2794181>
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.048796043>
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=-0.5486987>
 <tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([ 0.63676, -0.03045, -0.07589], dtype=float32)>
 <tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([-0.98515, -0.41528, -0.08934], dtype=float32)>
 <tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([0.63115, 0.39971, 0.58182], dtype=float32)>
 <tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([ 0.42818, -0.30391, -1.04652], dtype=float32)>]
[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.3274685>
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.08422458>
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=-0.55909735>
 <tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([ 0.65649, -0.41305,  0.33998], dtype=float32)>
 <tf.Variable 'Variable

C:\Users\krisw\AppData\Local\Temp\ipykernel_27088\3311764538.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(np.array(channel_model.parameter_list))
C:\Users\krisw\AppData\Local\Temp\ipykernel_27088\3311764538.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  print(np.array(channel_target.parameter_list))
